# Get NL-Augmenter and install dependencies

## Copy git repository of NL-Augmenter

In [1]:
!git clone https://www.github.com/GEM-benchmark/NL-Augmenter

Cloning into 'NL-Augmenter'...
remote: Enumerating objects: 11928, done.
remote: Counting objects: 100% (599/599), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 11928 (delta 334), reused 378 (delta 201), pack-reused 11329
Receiving objects: 100% (11928/11928), 91.24 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (7789/7789), done.


## Go to newly copied directory

In [2]:
cd NL-Augmenter

/content/NL-Augmenter


## Install requirements

In [3]:
!pip install -r requirements.txt --quiet

     |████████████████████████████████| 12.1 MB 240 kB/s 
     |████████████████████████████████| 12.7 MB 232 kB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 280 kB 60.6 MB/s 
     |████████████████████████████████| 234 kB 54.6 MB/s 
     |████████████████████████████████| 2.2 MB 69.9 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 138 kB 45.5 MB/s 
     |████████████████████████████████| 190 kB 59.4 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 105 kB 70.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.6 MB/s 
     |████████████████████████████████| 167 kB 67.6 MB/s 
     |████████████████████████████████| 451 kB 41.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████

# Data loading

##  Get dataloader and use it to load CommonGen dataset

In [4]:
from datasets import load_dataset

data = load_dataset("gem", "common_gen")

Dataset gem downloaded and prepared to /root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4. Subsequent calls will reuse this data.


## Explore data structure

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 67389
    })
    validation: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 993
    })
    test: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 1497
    })
    challenge_train_sample: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
    challenge_validation_sample: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
    challenge_test_scramble: Dataset({
        features: ['gem_id', 'gem_parent_id', 'concept_set_id', 'concepts', 'target', 'references'],
        num_rows: 500
    })
})

## Retrieve one entry from a train subset

In [6]:
train_instance = data['train'][0]
train_instance

{'concept_set_id': 0,
 'concepts': ['mountain', 'ski', 'skier'],
 'gem_id': 'common_gen-train-0',
 'gem_parent_id': 'common_gen-train-0',
 'references': [],
 'target': 'Skier skis down the mountain'}

## Retrieve a target sentence from this entry object

In [7]:
concept = train_instance.get("target")
concept

'Skier skis down the mountain'

# Test the filter

## Load a filter class from a corresponding ditectory and try the example from its ReadMe

In [8]:
from filters.gender_bias import GenderBiasFilter

f = GenderBiasFilter("en")
sentences = [ "He is going to make a cake.",
              "She is going to program",
              "Nobody likes washing dishes",
              "He agreed to help him" ]

f.filter(sentences)

True

## Take a subset of target sentences from CommonGem dataset 


In [14]:
def get_sentences(split_name, num_iter):
    sentences = []
    for i in range(num_iter):
        sentence = data[split_name][i].get("target")
        sentences.append(sentence)
    return sentences

sentences = get_sentences('train', 10)

## Apply a filter to this small subset

In [15]:
f.filter(sentences)

True

## Explore statistics which makes a filter returning the presence of gender bias

In [16]:
def get_stats(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine, masculine, neutral = f.count_genders(flagged_sentences)
    print("Feminine flagged sentences:", feminine)
    print("Masculine flagged sentences:", masculine)
    print("Neutral flagged sentences:", neutral)
    
get_stats(sentences, "en")

Feminine flagged sentences: 1
Masculine flagged sentences: 3
Neutral flagged sentences: 6


## Retrieve the sentences, corresponding to the statistics

In [17]:
def get_phrases(text, language):
    flagged_sentences = f.flag_sentences(text, language)
    feminine_group, masculine_group, neutral_group = f.sort_groups(flagged_sentences)    
    return feminine_group, masculine_group, neutral_group

feminine_group, masculine_group, neutral_group = get_phrases(sentences, "en")

print("This is a feminine group:", feminine_group)
print("This is a masculine group:", masculine_group)
print("This is a neutral group:", neutral_group)

This is a feminine group: ['woman paddling canoe on a lake']
This is a masculine group: ['The dog is wagging his tail.', 'A dog wags his tail at the boy.', 'a man paddles his canoe on the lake.']
This is a neutral group: ['Skier skis down the mountain', 'A skier is skiing down a mountain.', 'Three skiers are skiing on a snowy mountain.', 'a dog wags its tail with its heart', 'paddle an open canoe along lake .', 'a train pulls into station']


## Reduce the number of sentences from the majority group, to balance it with the minority group

In [18]:
reduced_masculine_group = masculine_group[2:3]
reduced_masculine_group

['a man paddles his canoe on the lake.']

## Create a balanced version of the subset

In [19]:
def create_balanced(split_name, num_iter, feminine_group, masculine_group, neutral_group):
    balanced = []
    balanced_sentences =  []

    for i in range(num_iter):
        train_instance = data[split_name][i].get("target")

        if train_instance in feminine_group:
            balanced.append(data[split_name][i])
            balanced_sentences.append(train_instance)
        elif train_instance in masculine_group:
            balanced.append(data[split_name][i])
            balanced_sentences.append(train_instance)
        elif train_instance in neutral_group:
            balanced.append(data[split_name][i])
            balanced_sentences.append(train_instance)

    print("The length of the balanced array is", len(balanced))
    return balanced, balanced_sentences

balanced, balanced_sentences = create_balanced("train", 10, feminine_group, reduced_masculine_group, neutral_group)

The length of the balanced array is 8


In [20]:
balanced_sentences

['Skier skis down the mountain',
 'A skier is skiing down a mountain.',
 'Three skiers are skiing on a snowy mountain.',
 'a dog wags its tail with its heart',
 'woman paddling canoe on a lake',
 'paddle an open canoe along lake .',
 'a man paddles his canoe on the lake.',
 'a train pulls into station']

In [21]:
balanced

[{'concept_set_id': 0,
  'concepts': ['mountain', 'ski', 'skier'],
  'gem_id': 'common_gen-train-0',
  'gem_parent_id': 'common_gen-train-0',
  'references': [],
  'target': 'Skier skis down the mountain'},
 {'concept_set_id': 0,
  'concepts': ['mountain', 'ski', 'skier'],
  'gem_id': 'common_gen-train-1',
  'gem_parent_id': 'common_gen-train-1',
  'references': [],
  'target': 'A skier is skiing down a mountain.'},
 {'concept_set_id': 0,
  'concepts': ['mountain', 'ski', 'skier'],
  'gem_id': 'common_gen-train-2',
  'gem_parent_id': 'common_gen-train-2',
  'references': [],
  'target': 'Three skiers are skiing on a snowy mountain.'},
 {'concept_set_id': 1,
  'concepts': ['dog', 'tail', 'wag'],
  'gem_id': 'common_gen-train-5',
  'gem_parent_id': 'common_gen-train-5',
  'references': [],
  'target': 'a dog wags its tail with its heart'},
 {'concept_set_id': 2,
  'concepts': ['canoe', 'lake', 'paddle'],
  'gem_id': 'common_gen-train-6',
  'gem_parent_id': 'common_gen-train-6',
  'refere

## Verify that the filter returns "False" with the balanced selection of sentences

In [22]:
f.filter(balanced_sentences)

False

# Apply the process to multiple entries

In [23]:
sentences = get_sentences('validation', len(data["validation"]))
f.filter(sentences)

True

In [24]:
get_stats(sentences, "en")

Feminine flagged sentences: 192
Masculine flagged sentences: 344
Neutral flagged sentences: 457


In [77]:
feminine_group, masculine_group, neutral_group = get_phrases(sentences, "en")
reduced_masculine_group = masculine_group[:192]

In [78]:
balanced, balanced_sentences = create_balanced('validation', len(data["validation"]), feminine_group, reduced_masculine_group, neutral_group)

The length of the balanced array is 841


## Wrap a new subset to a Dataset object: https://github.com/huggingface/datasets/blob/master/templates/new_dataset_script.py